In [1]:
import requests
import json
import re

def get_chunk(query: str,agent_id: str):
    chat_query_body = {
        "query": query,
        "rationale": "To get the product information.",
        "chat_history": [],
        "custom_field": {
            "agent_id": agent_id,
            "pru_force_lang": "en"
        }
    }

    headers = {
        'Ocp-Apim-Subscription-Key': '123'
    }

    #Call chat query
    chat_query_response = requests.post(
        "https://qaflow-api-svc-hklife-dev-az2-ai-pil.lb1-hklife-uat-az2-o8crb5.pru.intranet.asia/chat/query",
        headers=headers,
        json=chat_query_body,
        verify=False
    ).json()
    print(chat_query_response)

    query_str = chat_query_body["query"]
    response_str = chat_query_response.get("response", "")
    qaflow_process_id = chat_query_response.get("id", "")

    if not chat_query_response.get("citations"):
        final_result = {
            "query":             query_str,
            "response":          response_str,
            "keyword_knowledges": [],
            "vector_citations":   [],
            "graph_citations":    [],
            "rerank_citations":   []
        }
        print(final_result)
        return final_result

    system_query_body = {
        "data_filter": {
            "string_filter": {
                "qaflow_process_id_filter": [qaflow_process_id]
            }
        }
    }

    system_query_response = requests.post(
        "https://qaflow-api-svc-hklife-dev-az2-ai-pil.lb1-hklife-uat-az2-o8crb5.pru.intranet.asia/system/qaflow_process_log/query",
        json=system_query_body,
        verify=False
    ).json()

    if len(system_query_response.get("filtered_data", [])) == 0:
        final_result = {
            "query":             query_str,
            "response":          response_str,
            "keyword_knowledges": [],
            "vector_citations":   [],
            "graph_citations":    [],
            "rerank_citations":   []
        }
        print(final_result)
        return final_result

    knowledge_retrieval = system_query_response["filtered_data"][0]["knowledge_retrieval"]

    
    all_citations = (
        knowledge_retrieval.get("keyword_knowledges", []) +
        knowledge_retrieval.get("vector_citations", []) +
        knowledge_retrieval.get("graph_citations", []) +
        knowledge_retrieval.get("rerank_citations", [])
    )

    unique_data_ids = list({c["data_id"] for c in all_citations if c.get("data_id")})
    unique_knowledge_ids = list({c["knowledge_id"] for c in all_citations if c.get("knowledge_id")})

    data_id_to_raw_data = {}
    if unique_data_ids:
        vector_req_body = {
            "vector_filter": {
                "string_filter": {
                    "data_id_filter": unique_data_ids
                },
                "numeric_filter": {
                    "data_status_min": 1 
                }
            }
        }
        try:
            vector_response = requests.post(
                "https://data-pipe-svc-hklife-uat-az2-ai-pil.lb1-hklife-uat-az2-o8crb5.pru.intranet.asia/system/vector/query",
                json=vector_req_body,
                verify=False
            ).json()

            filtered_vectors = vector_response.get("filtered_vectors") or []
            for fv in filtered_vectors:
                d_id = fv.get("data_id")
                r_data = fv.get("raw_data", "")
                data_id_to_raw_data[d_id] = r_data

        except Exception as e:
            print(f"[batch_vector_query] Vector query failed: {e}")


    knowledge_id_to_name = {}
    if unique_knowledge_ids:
        knowledge_req_body = {
            "data_filter": {
                "string_filter": {
                    "knowledge_id_filter": unique_knowledge_ids
                },
                "numeric_filter": {
                    "knowledge_status_min": 1  
                }
            }
        }
        try:
            knowledge_response = requests.post(
                "https://data-pipe-svc-hklife-uat-az2-ai-pil.lb1-hklife-uat-az2-o8crb5.pru.intranet.asia/system/knowledge/query",
                json=knowledge_req_body,
                verify=False
            ).json()

            filtered_data = knowledge_response.get("filtered_data") or []
            for fd in filtered_data:
                k_id = fd.get("knowledge_id")
                k_name = fd.get("knowledge_name", "")
                knowledge_id_to_name[k_id] = k_name

        except Exception as e:
            print(f"[batch_knowledge_query] Knowledge query failed: {e}")

 
    def transform_citation(raw_log: dict) -> dict:
        data_id = raw_log.get("data_id", "")
        knowledge_id = raw_log.get("knowledge_id", "")
        data_version = raw_log.get("data_version", "")

        new_raw_data = data_id_to_raw_data.get(data_id, "") if data_id else ""
        new_knowledge_name = knowledge_id_to_name.get(knowledge_id, "") if knowledge_id else ""

        return {
            "data_id":         data_id,
            "data_version":    data_version,
            "knowledge_id":    knowledge_id,
            "raw_data":        new_raw_data,
            "knowledge_name":  new_knowledge_name
        }

    keyword_knowledges_ = [transform_citation(item) for item in knowledge_retrieval.get("keyword_knowledges", [])]
    vector_citations_   = [transform_citation(item) for item in knowledge_retrieval.get("vector_citations", [])]
    graph_citations_    = [transform_citation(item) for item in knowledge_retrieval.get("graph_citations", [])]
    rerank_citations_   = [transform_citation(item) for item in knowledge_retrieval.get("rerank_citations", [])]

   
    final_result = {
        "query":              query_str,
        "response":           response_str,
        "keyword_knowledges": keyword_knowledges_,
        "vector_citations":   vector_citations_,
       "graph_citations":    graph_citations_,
        "rerank_citations":   rerank_citations_
    }

    print(final_result)
    return final_result

In [2]:
get_chunk(query="需要填妥啲咩資料？",agent_id="Cashier")

c:\Users\npithkw\AppData\Local\miniconda3\Lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qaflow-api-svc-hklife-dev-az2-ai-pil.lb1-hklife-uat-az2-o8crb5.pru.intranet.asia'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'id': '8cbc9245-febb-4bd6-a24f-b7bbf8f4373c', 'response': "I'm sorry, but I can't answer your question as it might not be related to our company's services. Could you please ask something about our products, underwriting, or other insurance-related topics?", 'citations': []}
{'query': '需要填妥啲咩資料？', 'response': "I'm sorry, but I can't answer your question as it might not be related to our company's services. Could you please ask something about our products, underwriting, or other insurance-related topics?", 'keyword_knowledges': [], 'vector_citations': [], 'graph_citations': [], 'rerank_citations': []}


{'query': '需要填妥啲咩資料？',
 'response': "I'm sorry, but I can't answer your question as it might not be related to our company's services. Could you please ask something about our products, underwriting, or other insurance-related topics?",
 'keyword_knowledges': [],
 'vector_citations': [],
 'graph_citations': [],
 'rerank_citations': []}